# Apache data example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/apache_data_example.ipynb)

In [1]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [3]:
# inputs

# netdata demo space https://app.netdata.cloud/spaces/netdata-demo/rooms/apache/overview
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' 
room_id = '9115ab73-317f-4a24-9233-f8d2993dbfc1'
contexts = [
    'apache.requests',
    'apache.connections',
    'apache.conns_async',
    'apache.net',
    'apache.scoreboard',
    'apache.workers'
    ]

# last 10 minutes of data
after = -60*10
before = 0
freq = '10s' # 10 seconds

In [20]:
df = pd.DataFrame(columns=['time'])

# get data from netdata cloud
for context in contexts:
    df_context = get_data_cloud(space_id, room_id, context, after, before, freq=freq)
    df_context = df_context.add_prefix(f'{context}|')
    df = df.merge(df_context,how='outer',on='time')

df_time_min = df.index.min()
df_time_max = df.index.max()

# save tokens by using t
df['t'] = range(len(df))
df = df.set_index('t')
del df['time']
# try save tokens by using ints
df = (df*100).round(0).ffill().astype('int')
print(df.shape)
df.head()

(59, 12)


,apache.requests|requests,apache.connections|connections,apache.conns_async|closing,apache.conns_async|keepalive,apache.net|sent,apache.scoreboard|logging,apache.scoreboard|open,apache.scoreboard|reading,apache.scoreboard|sending,apache.scoreboard|waiting,apache.workers|busy,apache.workers|idle
t,,,,,,,,,,,,
0,200,200,0,60,1841,10,30000,0,120,9870,130,9870
1,150920,210,0,140,54881,0,30000,0,130,9870,130,9870
2,249640,210,0,150,89520,0,30000,0,110,9890,110,9890
3,136210,200,0,190,49758,10,30000,0,110,9880,120,9880
4,190,200,0,200,1762,0,30000,10,120,9870,130,9870


In [25]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring an apache web server using Netdata Cloud.

Below is the some apache metrics for this server from {df_time_min} to {df_time_max}, aggregated to a {freq} frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does this server have a problem?

Metrics data:
{df.to_string(max_rows=len(df), max_cols=(len(df.columns)))}
"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring an apache web server using Netdata Cloud.

Below is the some apache metrics for this server from 0 to 58, aggregated to a 10s frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does this server have a problem?

Metrics data:
    apache.requests|requests  apache.connections|connections  apache.conns_async|closing  apache.conns_async|keepalive  apache.net|sent  apache.scoreboard|logging  apache.scoreboard|open  apache.scoreboard|reading  apache.scoreboard|sending  apache.scoreboard|waiting  apache.workers|busy  apache.workers|idle
t                                                                                                                                                                                                                                                                                                                    
0                        

In [26]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [27]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [29]:
reply_content = completion.choices[0].message.content
pp.pprint(reply_content)

('Based on the provided metrics, it does not appear that the server has a '
 'problem. The number of requests and connections are fluctuating, but that is '
 'normal for a web server. The Apache workers are also staying busy, which is '
 'expected. The metrics for sent data and scoreboard status are also within '
 'reasonable ranges. Overall, there is no clear indication of any issues with '
 'this server.')
